In [1]:
 class Node:
    def __init__(self, index, demand, serv_time, time_window, lat, long):
        self.index = index
        self.demand = demand
        self.serv_time = serv_time
        self.time_window = time_window
        self.lat = lat
        self.long = long

    def __index__(self):
        return self.index

    def __hash__(self):
        return self.index

    def __repr__(self):
        return f"Customer <{self.index}>"

In [2]:
from docplex.mp.model import Model

def build_VRPTW_problem(capacity, nodes, times, vehicles,**kwargs):
    
    
    #Parámetros
    customers = [cstmrs for cstmrs in nodes if cstmrs.index != 0] 
    #Big M
    bigM = 1000

    #Declaración del modelo
    m = Model(name = 'VRPTW', **kwargs)
    
    # ----- Variables -----
    #Arco cubierto por vehículo
    m.x_var = m.binary_var_cube(vehicles,nodes,nodes, name= 'route')
    #Start time for the service of a customer
    m.s_var = m.continuous_var_matrix(vehicles, nodes, name = 'service_time')

    # ----- Restricciones -----
    #La satisfacción del cliente
    m.covering_cts = []
    for i in customers: 
        cust_satisf_ct = m.sum(
            m.x_var[u,i,j] for j in nodes for u in vehicles)>= 1
        cust_satisf_ct.name = 'customer_satisf_{0!s}'.format(i.index)
        m.covering_cts.append(cust_satisf_ct)

    m.add_constraints(m.covering_cts)
     
    # Estructura de ruta
    m.flow_cts = []
    for i in nodes:
        for u in vehicles:
            flow_ct = (m.sum(m.x_var[u,i,j] for j in nodes) - 
                m.sum(m.x_var[u,j,i] for j in nodes)) == 0
            flow_ct.name = 'flow_constraint_{0!s}_{1!s}'.format(i.index,u)
            m.flow_cts.append(flow_ct)

    for u in vehicles: 
        flow_ct = m.sum(m.x_var[u,nodes[0],i] for i in nodes) <= 1
        flow_ct.name = 'flow_constraint_{0!s}_{1!s}_origin'.format(nodes[0].index,u)
        m.flow_cts.append(flow_ct)

    m.add_constraints(m.flow_cts)
 
    #Capacidad del vehículo
    m.capacity_cts = []
    for u in vehicles:
        capacity_ct = m.sum(nodes[i].demand * m.x_var[u,i,j] for i in nodes for j in nodes) <= capacity
        capacity_ct.name = 'capacity_vehicle{0!s}'.format(u)
        m.capacity_cts.append(capacity_ct)
    
    m.add_constraints(m.capacity_cts)
    
    #Ventanas de tiempo
    m.time_window_cts = []
    for i in nodes:
        for j in customers: 
            for u in vehicles:
             time_window_ct = m.s_var[u,i] + i.serv_time + times[i.index][j.index] - m.s_var[u,j] + bigM*m.x_var[u,i,j] <= bigM
             time_window_ct.name = 'time_window_{0!s}_{1!s}_{2!s}'.format(u,i.index,j.index)
             m.time_window_cts.append(time_window_ct)
    
    for i in customers:
        for u in vehicles: 
         time_window_ct = m.s_var[u,i]+ i.serv_time + times[i][0] - nodes[0].time_window[1] + bigM*m.x_var[u,i,nodes[0]]<= bigM
         time_window_ct.name = 'time_window_{0!s}_{1!s}_{2!s}_arrival'.format(u,i.index,nodes[0].index)
         m.time_window_cts.append(time_window_ct)

    for i in nodes:
        for u in vehicles:
            time_window_ct_lb = i.time_window[0] <= m.s_var[u,i]
            time_window_ct_lb.name = 'time_window_lb_{0!s}'.format(i.index)
            time_window_ct_ub = i.time_window[1] >= m.s_var[u,i]
            time_window_ct_ub.name = 'time_window_ub_{0!s}'.format(i.index)
            m.time_window_cts.append(time_window_ct_lb)
            m.time_window_cts.append(time_window_ct_ub)
    m.add_constraints(m.time_window_cts)
 
    #----- Función objetiva -----
    #Minimice el costo total
    m.total_cost = m.sum(times[i][j]*m.x_var[u,i,j] for i in nodes for j in nodes for u in vehicles)
    m.minimize(m.total_cost)
    
    return m


In [3]:
CENTRO_REPARTO = Node(0,0,0,[0,120],-12.055286215862173, -77.03761770324066)# Wilson centro cómputo compras
Punto_1 = Node(1,10,3,[0,120],-11.860864774319099, -77.08270885357037)# ,Domicilio 1 Puente Piedra
Punto_2 = Node(2,20,3,[0,120], -11.880264510131493, -77.02095950290273)# ,Domicilio 2 Carabayllo 
Punto_3 = Node(3,30,3,[0,120],-11.98956118254249, -77.09880073186692)# Domicilio 3 San Martin de Porres
Punto_4 = Node(4,20,3,[0,120], -12.047362913132519, -77.12649612188682)# ,Domicilio 4  Callao
Punto_5 = Node(5,20,3,[0,120],-12.165407596126173, -76.96328741326177)#Domicilio 5 Miraflores
Punto_6 = Node(6,65,3,[0,120],-12.022785437170445, -76.89412395721087)#Domicilio 6 Ate
Punto_7 = Node(7,15,3,[0,120],-12.114866761848441, -77.03982184054135)#Domicilio 7 Miraflores
Punto_8 = Node(8,40,3,[0,120],-12.243772882472243, -76.91589482960883)#Domicilio 8 Villa el Salvador
vehicles = [1,2,3]
capacity = 100
nodes = [CENTRO_REPARTO, Punto_1,Punto_2, Punto_3,Punto_4,Punto_5,Punto_6,Punto_7,Punto_8]

In [4]:
import folium
from folium import Marker

m = folium.Map(location=[-12.047209, -77.069313], tiles='openstreetmap', zoom_start=11)
for node in [n for n in nodes if n.index != 0]:
 Marker(location= [node.lat, node.long], icon= (folium.Icon(color='red', icon='home', prefix='fa'))).add_child(
    folium.Popup('Punto {}'.format(node.index))).add_to(m)

 Marker(location= [nodes[0].lat,nodes[0].long], icon= (folium.Icon(color='green', icon='truck', prefix='fa'))).add_child(
    folium.Popup('ZONA DE REPARTO')).add_to(m)
m

In [5]:

times = [[round(((node_origin.lat - node_destin.lat)**2 + 
    (node_origin.long - node_destin.long)**2)**(1/2),3)*100 
    for node_destin in nodes] for node_origin in nodes]
times

[[0.0,
  20.0,
  17.599999999999998,
  9.0,
  8.9,
  13.3,
  14.7,
  6.0,
  22.400000000000002],
 [20.0, 0.0, 6.5, 13.0, 19.2, 32.7, 24.9, 25.8, 41.8],
 [17.599999999999998,
  6.5,
  0.0,
  13.4,
  19.8,
  29.099999999999998,
  19.1,
  23.5,
  37.8],
 [9.0, 13.0, 13.4, 0.0, 6.4, 22.2, 20.7, 13.8, 31.3],
 [8.9,
  19.2,
  19.8,
  6.4,
  0.0,
  20.1,
  23.400000000000002,
  11.0,
  28.799999999999997],
 [13.3, 32.7, 29.099999999999998, 22.2, 20.1, 0.0, 15.9, 9.2, 9.2],
 [14.7, 24.9, 19.1, 20.7, 23.400000000000002, 15.9, 0.0, 17.2, 22.2],
 [6.0, 25.8, 23.5, 13.8, 11.0, 9.2, 17.2, 0.0, 17.9],
 [22.400000000000002,
  41.8,
  37.8,
  31.3,
  28.799999999999997,
  9.2,
  22.2,
  17.9,
  0.0]]

In [6]:
mod = build_VRPTW_problem(capacity, nodes, times, vehicles)

In [7]:
import json 
sol = mod.solve()
print (sol)


solution for: VRPTW
objective: 128.2
route_1_Customer <0>_Customer <6>=1
route_1_Customer <6>_Customer <0>=1
route_2_Customer <0>_Customer <5>=1
route_2_Customer <5>_Customer <8>=1
route_2_Customer <7>_Customer <0>=1
route_2_Customer <8>_Customer <7>=1
route_3_Customer <0>_Customer <4>=1
route_3_Customer <1>_Customer <2>=1
route_3_Customer <2>_Customer <0>=1
route_3_Customer <3>_Customer <1>=1
route_3_Customer <4>_Customer <3>=1
service_time_1_Customer <2>=120.000
service_time_1_Customer <6>=14.700
service_time_2_Customer <4>=120.000
service_time_2_Customer <5>=13.300
service_time_2_Customer <7>=111.000
service_time_2_Customer <8>=90.100
service_time_3_Customer <1>=34.300
service_time_3_Customer <2>=43.800
service_time_3_Customer <3>=18.300
service_time_3_Customer <4>=8.900



In [8]:
sol_dict = json.loads(sol.export_as_json_string())

sol_dict['CPLEXSolution']['variables'][0:16]


[{'index': '6', 'name': 'route_1_Customer <0>_Customer <6>', 'value': '1.0'},
 {'index': '54', 'name': 'route_1_Customer <6>_Customer <0>', 'value': '1.0'},
 {'index': '86', 'name': 'route_2_Customer <0>_Customer <5>', 'value': '1.0'},
 {'index': '134', 'name': 'route_2_Customer <5>_Customer <8>', 'value': '1.0'},
 {'index': '144', 'name': 'route_2_Customer <7>_Customer <0>', 'value': '1.0'},
 {'index': '160', 'name': 'route_2_Customer <8>_Customer <7>', 'value': '1.0'},
 {'index': '166', 'name': 'route_3_Customer <0>_Customer <4>', 'value': '1.0'},
 {'index': '173', 'name': 'route_3_Customer <1>_Customer <2>', 'value': '1.0'},
 {'index': '180', 'name': 'route_3_Customer <2>_Customer <0>', 'value': '1.0'},
 {'index': '190', 'name': 'route_3_Customer <3>_Customer <1>', 'value': '1.0'},
 {'index': '201', 'name': 'route_3_Customer <4>_Customer <3>', 'value': '1.0'},
 {'index': '245',
  'name': 'service_time_1_Customer <2>',
  'value': '120.00000000000023'},
 {'index': '249',
  'name': 'se